In [1]:
import numpy as np
import math
np.set_printoptions(precision=3)
# prints _very_ small numbers (floating point errors)as 0
np.set_printoptions(suppress=True) 

# for plotting
from ipywidgets import interact
from bokeh.io import push_notebook, output_notebook, show
from bokeh.plotting import figure, show, output_notebook, output_file
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, Segment, DataRange1d, PanTool, WheelZoomTool, BoxSelectTool
)
output_notebook()

router_names = ["pacwoman", "spooky", "toc-man", "spunky", "tim", "orson", "noname", "pinky", "blinky", "clyde", "inky"]

def construct_n_edge(n):
    startpoint = np.array([0,0])   
    unit_vec = np.array([1,0])
    angle = 2*math.pi / n
    rotation_matrix = np.array([[math.cos(angle), -math.sin(angle)], 
                               [math.sin(angle),  math.cos(angle)]])
    # built n-edge
    n_edge = []
    rotated_vector = unit_vec
    for i in range(n):
        rotated_vector = rotation_matrix.dot(rotated_vector)
        n_edge.append(rotated_vector)

    return n_edge

Loading BokehJS ...

In [14]:
def plot_n_edge(n_edge):
    x = []
    y = []
    
    # append first vector after last to close the circle (plotting detail)
    n_edge.append(n_edge[0])
    # rearrange vectors in x,y list
    for vec in n_edge:
        x.append(vec[0])
        y.append(vec[1])
        
    def plot(x,y): 
        p = figure(title="dots", plot_height=300, plot_width=300)
        r = p.line(x, y, color="#2222aa", line_width=3)
        x_text, y_text = zip(*n_edge)
        p.text(x_text, y_text, router_names[:len(x)-1], text_color="#333333", text_align="center", text_font_size="10pt")
        show(p)
        
    plot(x,y)    

# test generating and plotting
for n in range(3,9):
    n_edge = construct_n_edge(n)  
    plot_n_edge(n_edge)

In [3]:
# source http://bokeh.pydata.org/en/0.11.1/docs/user_guide/geo.html
anchor_lat = 49.028404  #degrees
anchor_long = 12.062200 #degrees
anchor_point = np.array([anchor_long, anchor_lat])
size_unit = 40 # edge length in meters
def point_to_coordinates(anchor_point, size_unit, point):
    r_earth = 6371000 # meters    
    u_earth = 2*math.pi*r_earth
    scale_lat = 360/u_earth*size_unit
    anchor_lat = anchor_point[1] # longitudinal scaling depends on lateral position on earth 
    scale_long = scale_lat/math.cos(math.radians(anchor_lat)) 
    scale_vec = np.array([scale_long, scale_lat])
    return point*scale_vec+anchor_point

# test conversion
np.set_printoptions(precision=6)
test_points = construct_n_edge(11)
test_coords = []
new_point = anchor_point
for point in test_points:
    old_point = new_point
    new_point = point_to_coordinates(anchor_point, size_unit, point)
    test_coords.append(new_point)
    print("converting", point, new_point, new_point-old_point)
    # print("lat, long scaling", scale_lat, scale_long)

converting [ 0.841254  0.540641] [ 12.062662  49.028598] [ 0.000462  0.000194]
converting [ 0.415415  0.909632] [ 12.062428  49.028731] [-0.000234  0.000133]
converting [-0.142315  0.989821] [ 12.062122  49.02876 ] [-0.000306  0.000029]
converting [-0.654861  0.75575 ] [ 12.061841  49.028676] [-0.000281 -0.000084]
converting [-0.959493  0.281733] [ 12.061674  49.028505] [-0.000167 -0.000171]
converting [-0.959493 -0.281733] [ 12.061674  49.028303] [ 0.       -0.000203]
converting [-0.654861 -0.75575 ] [ 12.061841  49.028132] [ 0.000167 -0.000171]
converting [-0.142315 -0.989821] [ 12.062122  49.028048] [ 0.000281 -0.000084]
converting [ 0.415415 -0.909632] [ 12.062428  49.028077] [ 0.000306  0.000029]
converting [ 0.841254 -0.540641] [ 12.062662  49.02821 ] [ 0.000234  0.000133]
converting [ 1. -0.] [ 12.062749  49.028404] [ 0.000087  0.000194]


In [19]:
def show_points_on_map(anchor_point, list_of_points):
    
    map_options = GMapOptions(lat=anchor_point[1], lng=anchor_point[0], map_type="roadmap", zoom=18)

    plot = GMapPlot(
        x_range=DataRange1d(), y_range=DataRange1d(), map_options=map_options, title="Regensburg"
    )

    # make list out of vectors for plotting
    x_points = []
    y_points = []
    for vec in list_of_points:
        x_points.append(vec[0])
        y_points.append(vec[1])
        
    source = ColumnDataSource(
        data=dict(
            lon=x_points,
            lat=y_points,
        )
    )

    circle = Circle(x="lon", y="lat", size=15, fill_color="blue", fill_alpha=0.8, line_color=None)
    plot.add_glyph(source, circle)
    
    plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())
    # output_file("gmap_plot.html")

    show(plot)
  
show_points_on_map(anchor_point, test_coords)    

In [5]:
"""
uci set gluon-node-info.@location[0].latitude=53.834470
uci set gluon-node-info.@location[0].longitude=10.702518
uci commit gluon-node-info
/etc/init.d/alfred restart
"""
def assign_coordinates_routers(names, koordinates):
    ssh_str_lat = "uci set gluon-node-info.@location[0].latitude="
    ssh_str_lon = "uci set gluon-node-info.@location[0].longitude="
    ssh_str_commit = "uci commit gluon-node-info"
    ssh_srt_restart = "/etc/init.d/alfred restart"    
    
    output_list = []
    for i,name in enumerate(router_names):
        output_list.append(name)
        lat = str(koordinates[i][1])
        lon = str(koordinates[i][0])
        output_list.append(ssh_str_lat + str(lat))
        output_list.append(ssh_str_lon + str(lon))
        output_list.append(ssh_str_commit)
        output_list.append(ssh_srt_restart)
    
    return output_list
        
output_list = assign_coordinates_routers(router_names, test_coords) 
for line in output_list: print(line)

pacwoman
uci set gluon-node-info.@location[0].latitude=49.028598484
uci set gluon-node-info.@location[0].longitude=12.0626615373
uci commit gluon-node-info
/etc/init.d/alfred restart
spooky
uci set gluon-node-info.@location[0].latitude=49.0287312207
uci set gluon-node-info.@location[0].longitude=12.0624279093
uci commit gluon-node-info
/etc/init.d/alfred restart
toc-man
uci set gluon-node-info.@location[0].latitude=49.0287600671
uci set gluon-node-info.@location[0].longitude=12.0621219217
uci commit gluon-node-info
/etc/init.d/alfred restart
spunky
uci set gluon-node-info.@location[0].latitude=49.0286758648
uci set gluon-node-info.@location[0].longitude=12.0618407235
uci commit gluon-node-info
/etc/init.d/alfred restart
tim
uci set gluon-node-info.@location[0].latitude=49.0285053473
uci set gluon-node-info.@location[0].longitude=12.0616735929
uci commit gluon-node-info
/etc/init.d/alfred restart
orson
uci set gluon-node-info.@location[0].latitude=49.0283026527
uci set gluon-node-info.@